In [71]:
from collections import namedtuple
from enum import Enum

import regex as re


class State(Enum):
    INSIDE = 0
    OUTSIDE = 1
    def flip(self):
        if self == State.INSIDE:
            return State.OUTSIDE
        else:
            return State.INSIDE

def sign(x):
    if x > 0:
        return 1
    else:
        return -1

class Vec(namedtuple("Point", ["y", "x"])):
    def __add__(self, other):
        return Vec(self.y + other.y, self.x + other.x)
    
    def __sub__(self, other):
        return Vec(self.y - other.y, self.x - other.x)
    
    def __mul__(self, other):
        match other:
            case Vec(y, x):
                y_mul, x_mul = y, x
            case int() | float() as scalar:
                y_mul, x_mul = scalar, scalar
            case val:
                raise ValueError(f"Incorrect type of value {val}: {type(val)}")
        return Vec(self.y * y_mul, self.x * x_mul)
    
    def step(self):
        if self.y != 0:
            return Vec(sign(self.y), 0)
        else:
            return Vec(0, sign(self.x))


class Direction(Vec, Enum):
    UP = Vec(-1, 0)
    RIGHT = Vec(0, 1)
    DOWN = Vec(1, 0)
    LEFT = Vec(0, -1)

DIRECTION_LOOKUP = {
    "U": Direction.UP,
    "R": Direction.RIGHT,
    "D": Direction.DOWN,
    "L": Direction.LEFT
}

In [72]:
def print_map(map):
    for row in map:
        print(row)
m = {1: "#", 0: "."}

def print_map2(map):
    res =  "\n".join([
        "".join([m[col] for col in row])
        for row in map
    ])
    print(res)

In [73]:
instruction_regex = r"(.) (\d+) \((.*)\)"
with open("18/example.txt") as f:
    data = f.read().splitlines()
    instructions = []
    for row in data:
        direction, num_steps, color = re.match(instruction_regex, row).groups()
        direction = DIRECTION_LOOKUP[direction]
        num_steps = int(num_steps)
        move = direction * num_steps
        instructions.append((direction, move, color))
instructions

[(<Direction.RIGHT: Vec(y=0, x=1)>, Vec(y=0, x=6), '#70c710'),
 (<Direction.DOWN: Vec(y=1, x=0)>, Vec(y=5, x=0), '#0dc571'),
 (<Direction.LEFT: Vec(y=0, x=-1)>, Vec(y=0, x=-2), '#5713f0'),
 (<Direction.DOWN: Vec(y=1, x=0)>, Vec(y=2, x=0), '#d2c081'),
 (<Direction.RIGHT: Vec(y=0, x=1)>, Vec(y=0, x=2), '#59c680'),
 (<Direction.DOWN: Vec(y=1, x=0)>, Vec(y=2, x=0), '#411b91'),
 (<Direction.LEFT: Vec(y=0, x=-1)>, Vec(y=0, x=-5), '#8ceee2'),
 (<Direction.UP: Vec(y=-1, x=0)>, Vec(y=-2, x=0), '#caa173'),
 (<Direction.LEFT: Vec(y=0, x=-1)>, Vec(y=0, x=-1), '#1b58a2'),
 (<Direction.UP: Vec(y=-1, x=0)>, Vec(y=-2, x=0), '#caa171'),
 (<Direction.RIGHT: Vec(y=0, x=1)>, Vec(y=0, x=2), '#7807d2'),
 (<Direction.UP: Vec(y=-1, x=0)>, Vec(y=-3, x=0), '#a77fa3'),
 (<Direction.LEFT: Vec(y=0, x=-1)>, Vec(y=0, x=-2), '#015232'),
 (<Direction.UP: Vec(y=-1, x=0)>, Vec(y=-2, x=0), '#7a21e3')]

In [74]:
from functools import reduce

def calculate_bounds(current_vals: tuple[Vec, Vec, Vec], next_move: Vec) -> tuple[Vec, Vec, Vec]:
    current_position, bounds_min, bounds_max = current_vals
    current_position += next_move
    y_c, x_c = current_position
    bounds_min = Vec(min(y_c, bounds_min.y), min(x_c, bounds_min.x))
    bounds_max = Vec(max(y_c, bounds_max.y), max(x_c, bounds_max.x))

    return current_position, bounds_min, bounds_max

def prepare_bounds(instructions):
    starting_position, min_bounds, max_bounds = reduce(calculate_bounds,
       [instruction[1] for instruction in instructions],
       (Vec(0, 0), Vec(0, 0), Vec(0, 0))
    )
    y_min, x_min = min_bounds
    move = Vec(abs(y_min) if y_min < 0 else 0, abs(x_min) if x_min < 0 else 0)
    starting_position += move
    min_bounds += move
    max_bounds += move
    return starting_position, min_bounds, max_bounds

In [75]:
def prepare_ground(max_bounds):
    max_y, max_x = max_bounds
    return [
        [0 for x in range(max_x + 1)]
        for y in range(max_y+1)
    ]

In [76]:
starting_position, min_bounds, max_bounds = prepare_bounds(instructions)
starting_position, min_bounds, max_bounds

(Vec(y=0, x=0), Vec(y=0, x=0), Vec(y=9, x=6))

In [77]:
ground = prepare_ground(max_bounds)
print_map2(ground)

.......
.......
.......
.......
.......
.......
.......
.......
.......
.......


In [78]:
def dig_initial_tunnel(ground, instructions, starting_position):
    current_position = starting_position
    for _, move, _ in instructions:
        attr = "y" if move.y != 0 else "x"
            
        move_attr = getattr(move, attr)

        for _ in range(abs(move_attr)):
            current_y, current_x = current_position
            ground[current_y][current_x] = 1
            current_position += move.step()
    return ground
initial_tunnel = dig_initial_tunnel(ground, instructions, starting_position)
print_map2(initial_tunnel)

#######
#.....#
###...#
..#...#
..#...#
###.###
#...#..
##..###
.#....#
.######


In [79]:
from scipy.sparse import csr_matrix

def prepare_ground2(max_bounds):
    max_y, max_x = max_bounds
    return csr_matrix((max_y+1, max_x+1), dtype=int)


def dig_initial_tunnel2(ground, instructions, starting_position):
    current_position = starting_position
    for _, move, _ in instructions:
        destination = current_position + move
        # print(current_position, destination)
        y_c, x_c = current_position
        y_d, x_d = destination

        y_smaller, y_bigger = (y_c, y_d) if y_c < y_d else (y_d, y_c)
        x_smaller, x_bigger = (x_c, x_d) if x_c < x_d else (x_d, x_c)
        # print(y_c, y_d, x_c, x_d)

        ground[y_smaller:y_bigger+1, x_smaller:x_bigger+1] = 1
        current_position = destination
    return ground

ground2 = prepare_ground2(max_bounds)

In [80]:
from tqdm import tqdm

class Corner(Enum):
    UPPER_LEFT = 0
    UPPER_RIGHT = 1
    LOWER_LEFT = 2
    LOWER_RIGHT = 3

transitions = {
    (Corner.UPPER_LEFT, Corner.UPPER_RIGHT): False,
    (Corner.UPPER_LEFT, Corner.LOWER_RIGHT): True,
    (Corner.LOWER_LEFT, Corner.LOWER_RIGHT): False,
    (Corner.LOWER_LEFT, Corner.UPPER_RIGHT): True
}

def check_corner(ground: list[list[int]], y: int, x: int) -> Corner | None:
    row = ground[y]
    if row[x] != 1:
        return None

    if y < (len(ground) - 1) and x < (len(row) - 1) and row[x+1] == 1 and ground[y+1][x] == 1:
        return Corner.UPPER_LEFT
    

    if y < (len(ground) - 1) and x > 0 and row[x-1] == 1 and ground[y+1][x] == 1:
        return Corner.UPPER_RIGHT
    

    if y > 0 and x < (len(row) - 1) and row[x+1] == 1 and ground[y-1][x] == 1:
        return Corner.LOWER_LEFT

    if y > 0 and x > 0 and row[x-1] == 1 and ground[y-1][x] == 1:
        return Corner.LOWER_RIGHT

    
from copy import deepcopy

def dig(ground: list[list[int]]):
    res = deepcopy(ground)
    pbar = tqdm(total=len(ground))

    for y in range(len(ground)):
        pbar.update(1)
        state = State.OUTSIDE
        x = 0
        row = ground[y]

        def handle_corners(corner, y, x):
            next_hash_x = x+1
            while next_hash_x < (len(row)-1):
                if row[next_hash_x] == 1 and row[next_hash_x+1] == 0:
                    break

                else:
                    next_hash_x += 1

            if next_hash_x == (len(row)-1):
                should_continue = False
                should_transition = False
                next_x = -1
                return should_continue, should_transition, next_x
            else:
                next_corner = check_corner(ground, y, next_hash_x)
                if next_corner is None:
                    raise ValueError("Unhaled 1")
                
                should_transition = transitions[(corner, next_corner)] # type: ignore
                next_x = next_hash_x + 1
                should_continue = True
                return should_continue, should_transition, next_x
                
        
        while x < len(row):
            char = row[x]

            if x == (len(row) - 1):
                if state == State.INSIDE and char == 0:
                    res[y][x] = 1
                break

            if state == State.OUTSIDE:
                if char == 1:
                    corner = check_corner(ground, y, x)
                    if corner:
                        should_continue, should_transition, next_x = handle_corners(corner, y, x)
                        if not should_continue:
                            break

                        if should_transition:
                            state = state.flip()
                        x = next_x

                    else:
                        if row[x+1] == 0: # border
                            x += 1
                            state = State.INSIDE

                        else:
                            raise ValueError("Unhanled 222")
                else:
                    x += 1
            
            else:
                if char == 0:
                    res[y][x] = 1
                    x += 1
                
                else:
                    corner = check_corner(ground, y, x)
                    if corner:
                        should_continue, should_transition, next_x = handle_corners(corner, y, x)
                        if not should_continue:
                            break

                        if should_transition:
                            state = state.flip()
                        x = next_x
                    
                    else:
                        if row[x+1] == 0: # border
                            res[y][x] = 1
                            x += 1
                            state = State.OUTSIDE

                        else:
                            raise ValueError("Unhanled 333")

        
    return res


In [81]:
dug = dig(initial_tunnel)
print_map2(dug)

100%|██████████| 10/10 [00:00<00:00, 62045.92it/s]

#######
#######
#######
..#####
..#####
#######
#####..
#######
.######
.######


In [82]:
sum(sum(row) for row in dug)

62

## Part 2

In [83]:
instruction_regex = r"(.) (\d+) \(#(.*)\)"
with open("18/example.txt") as f:
    data = f.read().splitlines()
    instructions = []
    for row in data:
        direction, num_steps, color = re.match(instruction_regex, row).groups()
        num_steps = int(color[:5], 16)
        direction = {
            "0": "R",
            "1": "D",
            "2": "L",
            "3": "U"
        }[color[5]]


        
        direction = DIRECTION_LOOKUP[direction]
        move = direction * num_steps
        instructions.append((direction, move, color))
instructions

[(<Direction.RIGHT: Vec(y=0, x=1)>, Vec(y=0, x=461937), '70c710'),
 (<Direction.DOWN: Vec(y=1, x=0)>, Vec(y=56407, x=0), '0dc571'),
 (<Direction.RIGHT: Vec(y=0, x=1)>, Vec(y=0, x=356671), '5713f0'),
 (<Direction.DOWN: Vec(y=1, x=0)>, Vec(y=863240, x=0), 'd2c081'),
 (<Direction.RIGHT: Vec(y=0, x=1)>, Vec(y=0, x=367720), '59c680'),
 (<Direction.DOWN: Vec(y=1, x=0)>, Vec(y=266681, x=0), '411b91'),
 (<Direction.LEFT: Vec(y=0, x=-1)>, Vec(y=0, x=-577262), '8ceee2'),
 (<Direction.UP: Vec(y=-1, x=0)>, Vec(y=-829975, x=0), 'caa173'),
 (<Direction.LEFT: Vec(y=0, x=-1)>, Vec(y=0, x=-112010), '1b58a2'),
 (<Direction.DOWN: Vec(y=1, x=0)>, Vec(y=829975, x=0), 'caa171'),
 (<Direction.LEFT: Vec(y=0, x=-1)>, Vec(y=0, x=-491645), '7807d2'),
 (<Direction.UP: Vec(y=-1, x=0)>, Vec(y=-686074, x=0), 'a77fa3'),
 (<Direction.LEFT: Vec(y=0, x=-1)>, Vec(y=0, x=-5411), '015232'),
 (<Direction.UP: Vec(y=-1, x=0)>, Vec(y=-500254, x=0), '7a21e3')]

In [84]:
starting_position, min_bounds, max_bounds = prepare_bounds(instructions)
starting_position, min_bounds, max_bounds

(Vec(y=0, x=0), Vec(y=0, x=0), Vec(y=1186328, x=1186328))

In [85]:
ground = prepare_ground2(max_bounds)
ground.shape

(1186329, 1186329)

In [87]:
initial_tunnel = dig_initial_tunnel2(ground, instructions, starting_position)

/Users/szymon/projects/aoc23/env/lib/python3.10/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


<1186329x1186329 sparse matrix of type '<class 'numpy.int64'>'
	with 6405262 stored elements in Compressed Sparse Row format>

In [ ]:
from tqdm import tqdm
from copy import deepcopy

def dig2(ground: list[list[int]]):
    result = 0
    pbar = tqdm(total=len(ground))

    for y in range(len(ground)):
        pbar.update(1)
        state = State.OUTSIDE
        x = 0
        row = ground[y]

        def handle_corners(corner, y, x):
            next_hash_x = x+1
            while next_hash_x < (len(row)-1):
                if row[next_hash_x] == 1 and row[next_hash_x+1] == 0:
                    break

                else:
                    next_hash_x += 1

            if next_hash_x == (len(row)-1):
                should_continue = False
                should_transition = False
                next_x = -1
                return should_continue, should_transition, next_x
            else:
                next_corner = check_corner(ground, y, next_hash_x)
                if next_corner is None:
                    raise ValueError("Unhaled 1")
                
                should_transition = transitions[(corner, next_corner)] # type: ignore
                next_x = next_hash_x + 1
                should_continue = True
                return should_continue, should_transition, next_x
                
        
        while x < len(row):
            char = row[x]

            if x == (len(row) - 1):
                if state == State.INSIDE and char == 0:
                    result += 1
                break

            if state == State.OUTSIDE:
                if char == 1:
                    corner = check_corner(ground, y, x)
                    if corner:
                        should_continue, should_transition, next_x = handle_corners(corner, y, x)
                        if not should_continue:
                            break

                        if should_transition:
                            state = state.flip()
                        x = next_x

                    else:
                        if row[x+1] == 0: # border
                            x += 1
                            state = State.INSIDE

                        else:
                            raise ValueError("Unhanled 222")
                else:
                    x += 1
            
            else:
                if char == 0:
                    res[y][x] = 1
                    x += 1
                
                else:
                    corner = check_corner(ground, y, x)
                    if corner:
                        should_continue, should_transition, next_x = handle_corners(corner, y, x)
                        if not should_continue:
                            break

                        if should_transition:
                            state = state.flip()
                        x = next_x
                    
                    else:
                        if row[x+1] == 0: # border
                            res[y][x] = 1
                            x += 1
                            state = State.OUTSIDE

                        else:
                            raise ValueError("Unhanled 333")

        
    return res
